In [1]:
###   Imports and Logging   ###
import sys
import logging
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import __version__ as pltver
import matplotlib.ticker as ticker
from skaero.atmosphere import coesa
from skaero import __version__ as skver


logging.basicConfig(
    level=logging.DEBUG,
    format=' %(asctime)s -  %(levelname)s -  %(message)s'
)

logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
logging.info('Python version: {}'.format(sys.version))
logging.info('Author: Benjamin Crews')
logging.info('Numpy version: {}'.format(np.version.version))
logging.info('Pandas version: {}'.format(pd.__version__))
logging.info('Matplotlib version: {}'.format(pltver))
logging.info('SciKit-Aero version: {}'.format(skver))
logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

 2020-02-14 07:20:11,325 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
 2020-02-14 07:20:11,328 -  INFO -  Python version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
 2020-02-14 07:20:11,329 -  INFO -  Author: Benjamin Crews
 2020-02-14 07:20:11,330 -  INFO -  Numpy version: 1.16.2
 2020-02-14 07:20:11,331 -  INFO -  Pandas version: 0.24.2
 2020-02-14 07:20:11,331 -  INFO -  Matplotlib version: 3.0.3
 2020-02-14 07:20:11,332 -  INFO -  SciKit-Aero version: 0.1
 2020-02-14 07:20:11,333 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
 2020-02-14 07:20:11,334 -  DEBUG -  Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
#####################
###   Functions   ###
#####################
def thrust_Ct_mom(T: float,
                  R: float,
                  rho: float,
                  Omega: float
                 ) -> float:
    '''
    Calculates the thrust coefficient based on momentum theory.
    
    Ensure consistent units are used!
    '''
    return T/(math.pi*R**2*rho*(Omega*R)**2)


def local_solidity(b: int,
                   c: float,
                   x: float,
                   R: float
                  ):
    '''
    Calculates the local solidity at blade station 'x' based on the blade geometry.
    '''
    # TODO: Lookup local chord based on x and chord function definition.
    # (c: function instead of c: float)
    return b*c/(math.pi*R)

In [3]:
#######################################
###   Input Rotor Characteristics   ###
#######################################
D = 35            # [ft]
R = D/2           # [ft]
b = 4             # [ ]
CHORD = 10.4/12   # [ft]
atm = coesa.table(0) # an atmosphere at 0 height
rho = atm[3]/515   # [slugs/ft3] ; coesa returns rho in SI units (kg/m3)/515 = slugs/ft3
OMEGA = 756/R     # [rad/sec]

THRUST = np.linspace(0, 5000, num=20)

In [7]:
# Initialize a dataframe with the thrust
df = pd.DataFrame(data=THRUST, columns=['Thrust [lbs]'])

f = open('results.txt', 'w')
write_data = f.write('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')
write_data = f.write('Python version: {}\n'.format(sys.version))
write_data = f.write('Author: Benjamin Crews\n')
write_data = f.write('Numpy version: {}\n'.format(np.version.version))
write_data = f.write('Pandas version: {}\n'.format(pd.__version__))
write_data = f.write('Matplotlib version: {}\n'.format(pltver))
write_data = f.write('SciKit-Aero version: {}\n'.format(skver))
write_data = f.write('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

In [8]:
print('First, Calculate Ct from Momentum Theory: Ct = {:.5f}, @ T = 5000\n'.format(thrust_Ct_mom(5000, R, rho, OMEGA)))
write_data = f.write('First, Calculate Ct from Momentum Theory: Ct = {:.5f}, @ T = 5000\n'.format(thrust_Ct_mom(5000, R, rho, OMEGA)))
df['Ct'] = df.apply(thrust_Ct_mom, args=(R, rho, OMEGA))


print('Next, Calculate the solidity, which is constant for a constant-chord blade:\n')
write_data = f.write('Next, Calculate the solidity, which is constant for a constant-chord blade:\n')
df['Solidity'] = local_solidity(b, CHORD, 1, R)


print('Next, Calculate the tip-loss factor: B = {:.5f}, @ T = 5000\n'.format(1-math.sqrt(2*0.003823)/b))
write_data = f.write('Next, Calculate the tip-loss factor: B = {:.5f}, @ T = 5000\n'.format(1-math.sqrt(2*0.003823)/b))
df['B'] = 1 - np.sqrt(2*df.Ct)/b


d_0 = 0.0087
d_1 = -0.0216
d_2 = 0.4
print('For the 3 drag coefficients, empirical values will be uses: {}, {}, {}\n'.format(d_0, d_1, d_2))
write_data = f.write('For the 3 drag coefficients, empirical values will be uses: {}, {}, {}\n'.format(d_0, d_1, d_2))
print('Finally, Cq can be calculated!\n')
write_data = f.write('Finally, Cq can be calculated!\n')
# Note, this calculation is for hover only. Vclimb instances are hardcoded to zero.
df['Cq'] = 0.5 * df.Ct * np.sqrt(0**2 + 2*df.Ct/df.B**2) + 0.5*0 + df.Solidity*d_0/8 \
            + 2*d_1*df.Ct/(3*2*math.pi*df.B**2) + (4*d_2/(df.Solidity*2*math.pi))*(df.Ct/df.B)**2


print('And the last value to be calculated is the Figure of Merit, M:\n')
write_data = f.write('And the last value to be calculated is the Figure of Merit, M:\n')
df['M'] = (1/math.sqrt(2))*(df.Ct**(3/2)/df.Cq)


print(df)
# write_data = f.write(df)

f.close()

First, Calculate Ct from Momentum Theory: Ct = 0.00382, @ T = 5000

Next, Calculate the solidity, which is constant for a constant-chord blade:

Next, Calculate the tip-loss factor: B = 0.97814, @ T = 5000

For the 3 drag coefficients, empirical values will be uses: 0.0087, -0.0216, 0.4

Finally, Cq can be calculated!

And the last value to be calculated is the Figure of Merit, M:

    Thrust [lbs]        Ct  Solidity         B        Cq         M
0       0.000000  0.000000  0.063056  1.000000  0.000069  0.000000
1     263.157895  0.000201  0.063056  0.994985  0.000070  0.028705
2     526.315789  0.000402  0.063056  0.992908  0.000074  0.077079
3     789.473684  0.000604  0.063056  0.991314  0.000079  0.132327
4    1052.631579  0.000805  0.063056  0.989970  0.000086  0.188446
5    1315.789474  0.001006  0.063056  0.988786  0.000093  0.242044
6    1578.947368  0.001207  0.063056  0.987716  0.000102  0.291345
7    1842.105263  0.001408  0.063056  0.986732  0.000111  0.335604
8    2105.26

In [ ]:
# print('Now, blade angles can be calculated.\nFirst, inflow angle, phi, at the blade tip:\n')
# # Note, that these calculations are only for hover, so a zero replaces the Vclimb terms
# df['phi_t [rad]'] = 0.5 * np.sqrt(0**2 + 2*df.Ct/df.B**2) + 0.5*0
# df['phi_t [deg]'] = np.rad2deg(df['phi_t [rad]'])
# print(df)

# print('Values don\'t seem unreasonable, so we can carry on.')

In [ ]:
# print('And now, blade angle, theta, at the tip:')
# print('Note: 2pi is used for lift-curve slope (a).')
# df['theta_t [rad]'] = 0.5 * np.sqrt(0**2 + 2*df.Ct/df.B**2) + 4/(df.Solidity*2*math.pi)*(df.Ct/df.B**2)
# df['theta_t [deg]'] = np.rad2deg(df['theta_t [rad]'])
# print(df)

In [ ]:
################################
###   Plotting the Results   ###
################################
fig, ax = plt.subplots(figsize=(15,9))

# Add the data and color it
ax.plot(df.Cq, df.Ct, color='orange', label='Ideal Twist', marker='o', markersize='4')
ax.legend()

# Axis labels
ax.set_xlabel('Drag Coefficient, $C_Q$', fontsize=12)
ax.set_ylabel('Thrust Coefficient, $C_T$', fontsize=12)
ax.set_title('Thrust and Power\n', fontsize=18)

# Move the axis and its label to the top
# ax.xaxis.set_label_position('top')
# ax.xaxis.tick_top()

# Set the ticks
ax.xaxis.set_major_locator(ticker.AutoLocator())
ax.tick_params(which='minor', width=0.75, length=2.5)
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.tick_params(axis='both', which='both', direction='in')

# Set the grid lines
ax.grid(b=True, which='major', linestyle=':')
ax.grid(b=True, which='minor', linestyle=':', alpha=0.3)

# Set the data labels
for i,v in zip(df.Cq, df.Ct):
    label = '{:.5f}'.format(v)
    ax.annotate(label,
                (i,v),
                textcoords='offset points',
                xytext=(10,-5),
                ha='left',
                color='brown'
                )
    
fig.savefig('Blade-Element_Thrust_and_Power.png')


# Figure of Merit plot
fig, ax = plt.subplots(figsize=(15,9))

# Add the data and color it
ax.plot(df.Ct, df.M, color='orange', label='Ideal Twist', marker='o', markersize='4')
ax.legend()

# Axis labels
ax.set_xlabel('Thrust Coefficient, $C_T$', fontsize=12)
ax.set_ylabel('Figure of Merit, $M$', fontsize=12)
ax.set_title('Figure of Merit and Power\n', fontsize=18)

# Move the axis and its label to the top
# ax.xaxis.set_label_position('top')
# ax.xaxis.tick_top()

# Set the ticks
ax.xaxis.set_major_locator(ticker.AutoLocator())
ax.tick_params(which='minor', width=0.75, length=2.5)
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.tick_params(axis='both', which='both', direction='in')

# Set the grid lines
ax.grid(b=True, which='major', linestyle=':')
ax.grid(b=True, which='minor', linestyle=':', alpha=0.3)

# Set the data labels
for i,v in zip(df.Ct, df.M):
    label = '{:.3f}'.format(v)
    ax.annotate(label,
                (i,v),
                textcoords='offset points',
                xytext=(0,3),
                ha='right',
                color='brown'
                )
    
fig.savefig('Blade-Element_Merit_and_Thrust.png')